In [1]:
import pandas as pd
import csv
import urllib
import json

In [2]:
target_url = 'https://fragalysis.diamond.ac.uk/api/targets/?title=Mpro'

with urllib.request.urlopen(target_url) as f:
            response = json.loads(f.read().decode('utf-8'))

In [3]:
metadata_file = response['results'][0]['metadata']

In [4]:
mf = metadata_file.replace('http', 'https')

In [5]:
meta_response = urllib.request.urlopen(mf)

lines = [l.decode('utf-8') for l in meta_response.readlines()]
cr = csv.reader(lines)

csv_list = [row for row in cr]

header = csv_list[0]
del csv_list[0]

df = pd.DataFrame(csv_list, columns=header)

structures_dict = {'SMILES':[], 'CID':[], 'structure_ID':[], 'structure_LINK':[], 'ORIGIN':[]}

for rname in df.index:
    row = df.loc[rname]
    if '-' in row['alternate_name']:
        crys_string = row['crystal_name'].split('-')[1].split('_')[0]
        structures_dict['SMILES'].append(row['smiles'])
        structures_dict['CID'].append(row['alternate_name'])
        structures_dict['structure_ID'].append(crys_string)
        structures_dict['structure_LINK'].append(f'https://fragalysis.diamond.ac.uk/viewer/react/preview/direct/target/Mpro/mols/{crys_string}/A')
        structures_dict['ORIGIN'].append('MOONSHOT') 

In [6]:
df2 = pd.DataFrame.from_dict(structures_dict)
df2.drop_duplicates(inplace=True)
df2.to_csv('fragalysis_structures_auto.csv', index=None)